# 🎯 Regressão Logística — Churn do Produto A
**Pós-Graduação BI & Analytics · ML Aplicado a Decisões de Negócio**

---

### 📡 Sinal de Negócio
> O gráfico de vendas mostrou que o **Produto A estabilizou** enquanto o **Produto B cresceu 90%** em 17 meses.  
> Isso gerou a pergunta: **quais clientes reduziram as compras do Produto A?**  
> Queremos identificar esses clientes *antes* que o churn seja irreversível.

### 🏗️ O que vamos construir
Um modelo de **classificação binária** que prevê a probabilidade de um cliente ter reduzido compras ≥ 20%.  
A saída é uma **lista ranqueada por risco** para o time comercial acionar.

---
### Roteiro
1. Importar e explorar a base
2. Pré-processamento e engenharia de features
3. Treinar o modelo
4. Avaliar (Matriz de Confusão, AUC-ROC)
5. Expected Maximum Profit (EMP)
6. Interpretar coeficientes e Odds Ratio
7. Gerar lista acionável para o CRM
8. Comparar com XGBoost (relações não-lineares)

In [ ]:
# ── CÉLULA PRONTA: imports e configuração ──────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, roc_curve, accuracy_score,
    f1_score, precision_score, recall_score
)
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

plt.rcParams.update({'figure.dpi': 120, 'font.family': 'DejaVu Sans',
                     'axes.spines.top': False, 'axes.spines.right': False})
print('✅ Bibliotecas carregadas')

In [ ]:
# ── CÉLULA PRONTA: carregar dados ──────────────────────────────────
# Altere a URL abaixo para o raw link do seu GitHub
URL = 'https://raw.githubusercontent.com/SEU_USUARIO/ml-bi-analytics-aula/main/data/clientes_produto_A.csv'

# Se estiver rodando localmente:
# df = pd.read_csv('clientes_produto_A.csv')

df = pd.read_csv(URL)
print(f'Shape: {df.shape}')
df.head()

## 1. Exploração da Base (EDA)

In [ ]:
# ── CÉLULA PRONTA: visão geral ─────────────────────────────────────
print('=== TIPOS E MISSINGS ===')
info = pd.DataFrame({
    'dtype': df.dtypes,
    'missing': df.isna().sum(),
    'missing_%': (df.isna().mean() * 100).round(1)
})
print(info.to_string())

print('\n=== TARGET ===')
vc = df['reduziu_compras'].value_counts()
print(f'Não reduziu (0): {vc[0]:,} ({vc[0]/len(df):.1%})')
print(f'Reduziu    (1): {vc[1]:,} ({vc[1]/len(df):.1%})')

In [ ]:
# ── CÉLULA PRONTA: distribuições ───────────────────────────────────
fig, axes = plt.subplots(2, 4, figsize=(16, 7))
axes = axes.flatten()

num_cols = ['var_compras_6m','freq_compras_trim','ticket_medio_ratio',
            'tempo_cliente_anos','nps_score','vendedor_rotatividade',
            'segmento_A','canal_digital']

cores = ['#c84b2f','#2f6ec8']
for i, col in enumerate(num_cols):
    for val, cor in zip([0,1], cores):
        subset = df[df['reduziu_compras'] == val][col].dropna()
        axes[i].hist(subset, bins=25, alpha=0.55, color=cor,
                     label=f'churn={val}', density=True)
    axes[i].set_title(col, fontsize=9)
    axes[i].legend(fontsize=7)

plt.suptitle('Distribuição das Features por Classe de Churn', fontweight='bold')
plt.tight_layout()
plt.show()

## 2. Pré-Processamento

In [ ]:
# ── CÉLULA PRONTA: encoding de variáveis categóricas ───────────────
df_model = df.copy()

# Dummies para região e porte (drop_first evita multicolinearidade perfeita)
df_model = pd.get_dummies(df_model, columns=['regiao', 'porte'], drop_first=True)

# Remover colunas que não entram no modelo
drop_cols = ['id_cliente', 'segmento', 'prob_churn_real']
df_model  = df_model.drop(columns=[c for c in drop_cols if c in df_model.columns])

print('Colunas após encoding:', df_model.shape[1])
print(df_model.dtypes.to_string())

In [ ]:
# ╔══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Tratamento de missing values   ║
# ╚══════════════════════════════════════════════════╝
# Dica: use SimpleImputer com strategy='median' para nps_score e ticket_medio_ratio
# Lembre: imputar ANTES de separar treino/teste ou DEPOIS?

# Separar X e y
TARGET = 'reduziu_compras'
X = df_model.drop(columns=[TARGET])
y = df_model[TARGET]

# Sua resposta:
imputer = SimpleImputer(strategy=___)
X_imp   = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

print(f'Missing após imputação: {X_imp.isna().sum().sum()}')

In [ ]:
# ╔═══════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Split treino/teste + normalização   ║
# ╚═══════════════════════════════════════════════════════╝
# Dica: use train_test_split com test_size=0.2, stratify=y, random_state=42
# Dica: StandardScaler — fit no treino, transform em ambos

X_train, X_test, y_train, y_test = train_test_split(
    ___, ___, test_size=___, stratify=___, random_state=42
)

scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = ___

print(f'Treino: {X_train.shape} | Teste: {X_test.shape}')
print(f'Churn no treino: {y_train.mean():.1%} | Churn no teste: {y_test.mean():.1%}')

## 3. Treinar o Modelo

In [ ]:
# ╔══════════════════════════════════════════╗
# ║  COMPLETE AQUI — Treinar LogisticRegression  ║
# ╚══════════════════════════════════════════╝
# Dica: LogisticRegression(C=1.0, solver='lbfgs', max_iter=500, random_state=42)
# Pergunta para o grupo: o que C controla?

logit = LogisticRegression(C=___, solver=___, max_iter=500, random_state=42)
logit.fit(___, ___)

y_pred_logit  = logit.predict(X_test)
y_proba_logit = logit.predict_proba(X_test)[:, 1]

print('✅ Modelo treinado')

## 4. Avaliação do Modelo

In [ ]:
# ╔══════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Matriz de confusão + métricas  ║
# ╚══════════════════════════════════════════════╝

cm  = confusion_matrix(___, ___)
auc = roc_auc_score(___, ___)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Heatmap da matriz de confusão
im = axes[0].imshow(cm, cmap='Blues')
for i in range(2):
    for j in range(2):
        axes[0].text(j, i, cm[i,j], ha='center', va='center',
                     fontsize=14, fontweight='bold',
                     color='white' if cm[i,j] > cm.max()/2 else 'black')
axes[0].set(xticks=[0,1], yticks=[0,1],
            xticklabels=['Pred 0','Pred 1'],
            yticklabels=['Real 0','Real 1'],
            title='Matriz de Confusão')

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_proba_logit)
axes[1].plot(fpr, tpr, color='#c84b2f', lw=2, label=f'AUC = {auc:.3f}')
axes[1].plot([0,1],[0,1],'--', color='#888', lw=1)
axes[1].set(xlabel='FPR', ylabel='TPR', title='Curva ROC')
axes[1].legend()
plt.tight_layout()
plt.show()

print(f'\nAcurácia : {accuracy_score(y_test, y_pred_logit):.3f}')
print(f'AUC-ROC  : {auc:.3f}')
print(f'F1-Score : {f1_score(y_test, y_pred_logit):.3f}')
print(f'Recall   : {recall_score(y_test, y_pred_logit):.3f}')
print(f'Precision: {precision_score(y_test, y_pred_logit):.3f}')

In [ ]:
# ── CÉLULA PRONTA: discussão FP vs FN ─────────────────────────────
print('╔══════════════════════════════════════════════════════╗')
print('║  DEBATE: o que é pior no nosso problema?             ║')
print('╠══════════════════════════════════════════════════════╣')
print('║  Falso Negativo (FN): não contactamos quem ia churnar║')
print('║  → Custo = CLV perdido                               ║')
print('║                                                      ║')
print('║  Falso Positivo (FP): contactamos quem não ia churnar║')
print('║  → Custo = custo da intervenção (desconto, ligação)  ║')
print('╚══════════════════════════════════════════════════════╝')
print()
print('Se CLV médio = R$ 2.000 e custo de contato = R$ 50:')
print(f'  FN: perdemos ~R$ {cm[1,0] * 2000:,}')
print(f'  FP: gastamos ~R$ {cm[0,1] * 50:,}')
print('\n→ Nesse contexto, minimizar FN (aumentar Recall) é prioritário.')

## 5. Expected Maximum Profit (EMP)

> O AUC-ROC não nos diz *onde cortar* a probabilidade. O EMP encontra o threshold que **maximiza o lucro esperado** dado o custo de cada tipo de erro.

In [ ]:
# ── CÉLULA PRONTA: EMP para churn e para default ──────────────────
from scipy.special import expit

def emp_churn(y_true, y_proba, clv=2000, custo_contato=50):
    """Threshold que maximiza lucro para retenção de clientes."""
    thresholds = np.linspace(0, 1, 300)
    lucros = []
    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        tp = ((y_pred == 1) & (y_true == 1)).sum()
        fp = ((y_pred == 1) & (y_true == 0)).sum()
        lucros.append(tp * (clv - custo_contato) - fp * custo_contato)
    idx = np.argmax(lucros)
    return thresholds, lucros, thresholds[idx], lucros[idx]

def emp_default(y_true, y_proba, valor_emp=15000, taxa_rec=0.30, custo_rej=300):
    """Threshold ótimo para aprovação de crédito."""
    thresholds = np.linspace(0, 1, 300)
    lucros = []
    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        tp = ((y_pred == 1) & (y_true == 1)).sum()
        fp = ((y_pred == 1) & (y_true == 0)).sum()
        lucros.append(tp * valor_emp * (1 - taxa_rec) - fp * custo_rej)
    idx = np.argmax(lucros)
    return thresholds, lucros, thresholds[idx], lucros[idx]

thr_c, luc_c, t_ot_c, emp_c = emp_churn(y_test, y_proba_logit)
thr_d, luc_d, t_ot_d, emp_d = emp_default(y_test, y_proba_logit)

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for ax, thr, luc, t_ot, emp_v, cor, titulo in [
    (axes[0], thr_c, luc_c, t_ot_c, emp_c, '#c84b2f', 'EMP — Churn de Produto'),
    (axes[1], thr_d, luc_d, t_ot_d, emp_d, '#2f6ec8', 'EMP — Crédito / Default')
]:
    ax.plot(thr, luc, color=cor, lw=2)
    ax.axvline(t_ot, color='#1a1a1a', lw=1.3, linestyle='--',
               label=f'Threshold ótimo: {t_ot:.2f}')
    ax.scatter([t_ot], [emp_v], color='#1a1a1a', s=60, zorder=5)
    ax.set(title=titulo, xlabel='Threshold', ylabel='Lucro Esperado (R$)')
    ax.legend(fontsize=9)
    ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'R${x:,.0f}'))

plt.suptitle('Mesmo AUC — thresholds diferentes porque os custos de negócio são diferentes',
             fontweight='bold', fontsize=10)
plt.tight_layout()
plt.show()

print(f'CHURN  → threshold ótimo: {t_ot_c:.2f} | EMP: R$ {emp_c:,.0f}')
print(f'DEFAULT → threshold ótimo: {t_ot_d:.2f} | EMP: R$ {emp_d:,.0f}')
print('\n💡 A métrica certa depende do problema — não existe uma resposta universal.')

## 6. Interpretação — Coeficientes e Odds Ratio

In [ ]:
# ╔═══════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Tabela de coeficientes + OR     ║
# ╚═══════════════════════════════════════════════════╝
# Dica: use statsmodels.api para obter p-values
# X_train_sm = sm.add_constant(X_train)
# logit_sm = sm.Logit(y_train, X_train_sm).fit()

X_cols = list(pd.get_dummies(
    df.drop(columns=['id_cliente','segmento','prob_churn_real','reduziu_compras'],
            errors='ignore'),
    columns=['regiao','porte'], drop_first=True
).columns)

X_train_sm = sm.add_constant(X_train)
logit_sm   = sm.Logit(y_train, X_train_sm).fit(disp=0)

coef_df = pd.DataFrame({
    'variavel':     ['intercepto'] + X_cols,
    'coeficiente':  logit_sm.params.values,
    'odds_ratio':   np.exp(logit_sm.params.values),
    'p_value':      logit_sm.pvalues.values
}).sort_values('odds_ratio', ascending=False)

coef_df['significancia'] = pd.cut(
    coef_df['p_value'],
    bins=[-1, 0.001, 0.01, 0.05, 1],
    labels=['***','**','*','ns']
)
print(coef_df.to_string(index=False, float_format=lambda x: f'{x:.4f}'))

In [ ]:
# ── CÉLULA PRONTA: gráfico de Odds Ratio ──────────────────────────
sig_df = coef_df[coef_df['variavel'] != 'intercepto'].copy()
sig_df = sig_df[sig_df['p_value'] < 0.05].sort_values('odds_ratio')

cores_or = ['#c84b2f' if or_ > 1 else '#2f6ec8' for or_ in sig_df['odds_ratio']]

fig, ax = plt.subplots(figsize=(9, max(5, len(sig_df) * 0.38)))
bars = ax.barh(sig_df['variavel'], sig_df['odds_ratio'] - 1,
               left=1, color=cores_or, alpha=0.8)
ax.axvline(1, color='#1a1a1a', lw=1.2, linestyle='--')
for bar, val in zip(bars, sig_df['odds_ratio']):
    ax.text(val + 0.02, bar.get_y() + bar.get_height()/2,
            f'{val:.2f}×', va='center', fontsize=9)
ax.set(xlabel='Odds Ratio', title='Odds Ratio — Variáveis Significativas (p < 0.05)')
plt.tight_layout()
plt.show()

print('\n💡 OR > 1 → aumenta probabilidade de churn')
print('   OR < 1 → protege contra churn')

## 7. Saída Acionável — Lista de Risco para o CRM

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Gerar lista ranqueada por risco    ║
# ╚══════════════════════════════════════════════════════╝
# Dica: calcule a probabilidade para TODA a base (não só o teste)
# Dica: use o threshold ótimo do EMP calculado acima (t_ot_c)

# Reconstituir X completo com as mesmas transformações
X_full = pd.get_dummies(df.drop(
    columns=['id_cliente','segmento','prob_churn_real','reduziu_compras'], errors='ignore'),
    columns=['regiao','porte'], drop_first=True)
X_full_imp = pd.DataFrame(imputer.transform(X_full), columns=X_full.columns)
X_full_sc  = scaler.transform(X_full_imp)

prob_full   = logit.predict_proba(X_full_sc)[:, 1]
flag_risco  = (prob_full >= ___).astype(int)   # ← use t_ot_c

lista_crm = pd.DataFrame({
    'id_cliente':      df['id_cliente'],
    'prob_churn':      prob_full.round(4),
    'alerta':          flag_risco,
    'segmento_A':      df['segmento_A'],
    'nps_score':       df['nps_score'],
    'canal_digital':   df['canal_digital'],
}).sort_values('prob_churn', ascending=False)

print(f'Clientes em alerta: {flag_risco.sum():,} ({flag_risco.mean():.1%} da base)')
print('\nTop 10 clientes prioritários:')
lista_crm.head(10)

In [ ]:
# ── CÉLULA PRONTA: exportar ────────────────────────────────────────
lista_crm.to_csv('lista_risco_crm.csv', index=False)
print('✅ lista_risco_crm.csv exportada')
print(f'   {flag_risco.sum():,} clientes para acionar')
print(f'   Concentração Seg. A: {lista_crm[lista_crm.alerta==1].segmento_A.mean():.1%}')

## 8. Comparação: XGBoost captura o que o Logit não vê

> O Logit assume que cada variável tem efeito **linear** na log-odds. O XGBoost não faz essa suposição.  
> O SHAP Dependence Plot vai mostrar que o NPS tem um efeito **não-linear**: indiferente acima de 5, catastrófico abaixo.

In [ ]:
# ── CÉLULA PRONTA: XGBoost no churn (código completo) ─────────────
try:
    from xgboost import XGBClassifier
    import shap

    xgb_clf = XGBClassifier(
        n_estimators=200, max_depth=4, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, eval_metric='logloss', verbosity=0
    )
    xgb_clf.fit(X_train, y_train)
    y_proba_xgb = xgb_clf.predict_proba(X_test)[:, 1]
    y_pred_xgb  = xgb_clf.predict(X_test)

    auc_logit = roc_auc_score(y_test, y_proba_logit)
    auc_xgb   = roc_auc_score(y_test, y_proba_xgb)

    print(f"{'Métrica':<20} {'Logit':>10} {'XGBoost':>10}")
    print('-' * 42)
    print(f"{'AUC-ROC':<20} {auc_logit:>10.3f} {auc_xgb:>10.3f}")
    print(f"{'Acurácia':<20} {accuracy_score(y_test, y_pred_logit):>10.3f} {accuracy_score(y_test, y_pred_xgb):>10.3f}")
    print(f"{'F1-Score':<20} {f1_score(y_test, y_pred_logit):>10.3f} {f1_score(y_test, y_pred_xgb):>10.3f}")

    # SHAP dependence plot — NPS não-linear
    explainer   = shap.TreeExplainer(xgb_clf)
    X_test_df   = pd.DataFrame(X_test, columns=X_cols)
    shap_values = explainer.shap_values(X_test_df)

    fig, axes = plt.subplots(1, 2, figsize=(13, 5))
    fpr_l, tpr_l, _ = roc_curve(y_test, y_proba_logit)
    fpr_x, tpr_x, _ = roc_curve(y_test, y_proba_xgb)
    axes[0].plot(fpr_l, tpr_l, label=f'Logit  (AUC={auc_logit:.3f})', color='#c84b2f', lw=2)
    axes[0].plot(fpr_x, tpr_x, label=f'XGBoost (AUC={auc_xgb:.3f})', color='#2f6ec8', lw=2)
    axes[0].plot([0,1],[0,1],'--', color='#888')
    axes[0].set(title='Curvas ROC — mesmo problema', xlabel='FPR', ylabel='TPR')
    axes[0].legend()

    shap.dependence_plot('nps_score', shap_values, X_test_df, ax=axes[1], show=False)
    axes[1].set_title('NPS × SHAP: relação não-linear\n(XGBoost vê; Logit não)')
    axes[1].axhline(0, color='#888', lw=0.8, linestyle='--')
    plt.tight_layout()
    plt.show()

    print('\n💡 Abaixo de NPS 5, o risco de churn acelera — relação não-linear.')
    print('   Logit vê um coeficiente médio. XGBoost captura o threshold crítico.')

except ImportError:
    print('⚠️  xgboost/shap não instalados. Execute: !pip install xgboost shap')
    print('   No Google Colab, ambos já estão disponíveis.')